In [3]:
from transformers import pipeline

In [20]:
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu


In [18]:
def analyse_sentiment(text: str) -> dict:
    result = sentiment_pipeline(text)

    # assign sentiment and confidence
    sentiment = result[0].get('label')
    confidence = result[0].get('score')

    return {
        "sentiment": sentiment,
        "confidence": confidence
    }


In [19]:
analyse_sentiment("This is amazing!")

{'sentiment': 'positive', 'confidence': 0.9747719764709473}

In [25]:
def analyse_emotions(text: str) -> dict:
    emotions = emotion_pipeline(text)

    top_emotions = sorted(emotions, key=lambda x : x['score'], reverse=True)[:3]

    return {
        "primary_emotion": top_emotions[0]['label'],
        "primary_confidence": top_emotions[0]['score'],
        "secondary_emotion": top_emotions[1]['label'] if len(top_emotions) > 1 else None,
        "all_emotions": top_emotions
    }

In [39]:
analyse_emotions("We're making a lot of progress!")

{'primary_emotion': 'joy',
 'primary_confidence': 0.6284446120262146,
 'secondary_emotion': None,
 'all_emotions': [{'label': 'joy', 'score': 0.6284446120262146}]}

In [40]:
def get_combined_analysis(text: str) -> dict:

    sentiment_data = analyse_sentiment(text)
    emotion_data = analyse_emotions(text)
    
    return {
        "sentiment": sentiment_data,
        "emotion": emotion_data,
        "text": text
    }

In [41]:
get_combined_analysis("We're making a lot of progress!")

{'sentiment': {'sentiment': 'positive', 'confidence': 0.9833697080612183},
 'emotion': {'primary_emotion': 'joy',
  'primary_confidence': 0.6284446120262146,
  'secondary_emotion': None,
  'all_emotions': [{'label': 'joy', 'score': 0.6284446120262146}]},
 'text': "We're making a lot of progress!"}